In [4]:
import os
os.chdir('D:\\MusaddiqueHussainLabs\\rcu_heritage_chatbot_demo\\data')

In [5]:
%pwd

'D:\\MusaddiqueHussainLabs\\rcu_heritage_chatbot_demo\\data'

In [6]:
import fitz  # PyMuPDF
import os
import json
from tqdm import tqdm
from langchain_core.documents import Document
from PIL import Image
import re

In [7]:
PDF_PATH = "alula-collections-100-objects.pdf"
OUTPUT_DIR = "output"
IMAGE_DIR = os.path.join(OUTPUT_DIR, "images")

In [8]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(IMAGE_DIR, exist_ok=True)

In [9]:
CANONICAL_FIELDS = {
    "Inv. No.": ["inv. no."],
    "Object title": ["object title"],
    "Artists / Makers": ["artists / makers", "artists /", "makers"],
    "Dimensions": ["dimensions"],
    "Material": ["material"],
    "Date": ["date"],
    "Description": ["description"],
    "Inscription Credit line": ["inscription credit line", "inscription", "credit line"],
    "Object Count": ["object count"]
}

EXCLUDE_KEYWORDS = ["museumplus", "RCU Collections"]

In [10]:
def normalize(text: str) -> str:
    return " ".join(text.lower().split())


def is_noise(text: str) -> bool:
    return any(k in normalize(text) for k in EXCLUDE_KEYWORDS)

In [11]:
def clean_field_value(field: str, value: str) -> str:
    """
    Removes duplicated field prefix like 'Material:' from value.
    """
    if not value:
        return value

    field_norm = field.lower().replace(" / ", " ")
    value_norm = value.lower()

    if value_norm.startswith(field_norm):
        return value.split(":", 1)[-1].strip()

    return value


In [12]:
def extract_images(page, inv_no):
    images = []

    for idx, img in enumerate(page.get_images(full=True)):
        xref = img[0]
        pix = fitz.Pixmap(page.parent, xref)

        if pix.n > 4:
            pix = fitz.Pixmap(fitz.csRGB, pix)

        width, height = pix.width, pix.height
        area = width * height

        # 🚫 FILTER 1: very small images (logos, icons)
        if area < 50_000:  # tweak if needed
            pix = None
            continue

        # 🚫 FILTER 2: extreme aspect ratio (museumplus barcode-like strip)
        aspect_ratio = max(width / height, height / width)
        if aspect_ratio > 8:
            pix = None
            continue

        path = os.path.join(IMAGE_DIR, f"{inv_no}_{len(images)+1}.png")
        pix.save(path)
        pix = None
        images.append(path)
    
    # ✅ DEFAULT IMAGE IF NONE FOUND
    # if not images:
    #     default_path = os.path.join(IMAGE_DIR, "RCU.2018.1.12_1.png")
    #     images = [default_path]

    return images

In [13]:
import re

def extract_table_metadata(page):
    blocks = page.get_text("blocks")
    blocks = sorted(blocks, key=lambda b: (b[1], b[0]))

    lines = []
    for b in blocks:
        text = b[4].strip()
        if not text or is_noise(text):
            continue
        for line in text.split("\n"):
            line = line.strip()
            if line:
                lines.append(line)

    metadata = {}
    i = 0

    while i < len(lines):
        raw_line = lines[i]
        current = normalize(raw_line)

        matched_field = None
        matched_pattern = None

        for field, patterns in CANONICAL_FIELDS.items():
            for p in patterns:
                if current.startswith(p):
                    matched_field = field
                    matched_pattern = p
                    break
            if matched_field:
                break

        if not matched_field:
            i += 1
            continue

        # 🔑 CASE 1: value exists on SAME line (Inv. No. .ﻋﻼ396.2)
        inline_value = raw_line[len(matched_pattern):].strip(" :")

        value_lines = []
        if inline_value:
            value_lines.append(inline_value)
            i += 1
        else:
            # 🔑 CASE 2: value on following lines
            i += 1
            while i < len(lines):
                next_norm = normalize(lines[i])

                # Stop at next field
                if any(
                    next_norm.startswith(p)
                    for pats in CANONICAL_FIELDS.values()
                    for p in pats
                ):
                    break

                value_lines.append(lines[i])
                i += 1

        value = " ".join(value_lines).strip()

        # ✅ Inv. No. normalization (English + Arabic)
        if matched_field == "Inv. No.":
            value = value.lstrip(".").strip()
            if value.startswith("علا") or value.startswith("ﻋﻼ"):
                value = re.sub(r"^[ﻋع]ﻼ\.?\s*", "علا ", value)

        metadata[matched_field] = value if value else None

    return metadata


In [14]:
def build_semantic_text(record):
    parts = []

    def add(label, value):
        if value and str(value).strip():
            parts.append(f"{label}: {value}")

    add("Inv. No.", record.get("inv_no"))
    add("Title", record.get("object_title"))
    add("Artists / Makers", record.get("artists_makers"))
    add("Material", record.get("material"))
    add("Date", record.get("date"))
    add("Dimensions", record.get("dimensions"))
    add("Object Count", record.get("object_count"))
    add("Inscription Credit Line", record.get("inscription_credit_line"))
    add("Description", record.get("description"))

    return " ".join(parts)


In [15]:
def ingest_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    objects, lc_docs = [], []

    last_record = None  # track last object for next-page images

    for i in tqdm(range(len(doc)), desc="Processing RCU pages"):
        page = doc[i]
        meta = extract_table_metadata(page)

        # ---------------------------
        # CASE 1: New Object Found
        # ---------------------------
        if "Inv. No." in meta:
            inv_no = meta["Inv. No."]

            images = extract_images(page, inv_no)

            record = {
                "inv_no": str(inv_no),
                "object_title": meta.get("Object title"),
                "artists_makers": meta.get("Artists / Makers"),
                "dimensions": meta.get("Dimensions"),
                "material": clean_field_value("Material", meta.get("Material")),
                "date": meta.get("Date"),
                "description": meta.get("Description"),
                "inscription_credit_line": meta.get("Inscription Credit line"),
                "object_count": meta.get("Object Count"),
                "images": images,
                "source": "RCU AlUla Collections - 100 Objects"
            }

            objects.append(record)
            last_record = record  # save reference

            semantic_text = build_semantic_text(record)

            lc_docs.append(
                Document(
                    page_content=semantic_text,
                    metadata={
                        "inv_no": inv_no,
                        "title": record["object_title"],
                        "artist": record["artists_makers"],
                        "dimensions": record["dimensions"],
                        "date": record["date"],
                        "material": record["material"],
                        "inscription_credit_line": record["inscription_credit_line"],
                        "object_count": record["object_count"],
                        "images": images,
                        "source": record["source"]
                    }
                )
            )

        # ---------------------------
        # CASE 2: Image-only page
        # ---------------------------
        else:
            if last_record:
                extra_images = extract_images(page, last_record["inv_no"])

                if extra_images:
                    last_record["images"].extend(extra_images)

                    # Also update corresponding LangChain doc metadata
                    for doc_obj in lc_docs:
                        if doc_obj.metadata.get("inv_no") == last_record["inv_no"]:
                            doc_obj.metadata["images"].extend(extra_images)
                            break

    return objects, lc_docs


In [16]:
objects, lc_docs = ingest_pdf(PDF_PATH)

Processing RCU pages: 100%|██████████| 125/125 [00:02<00:00, 43.58it/s]


In [17]:
objects

[{'inv_no': 'RCU.2018.1.12',
  'object_title': "Mada'in Salih",
  'artists_makers': 'Andreas Schmidt-Colinet',
  'dimensions': None,
  'material': 'reversal film; Material: plastic (material)',
  'date': '1982 AD',
  'description': "-> en: From Dr. A. Schmidt-Colinet collection of 141 colour 35mm slides from 1982 Hegra Excavations. It shows Mad'in Saleh tombs, Hejaz railway buildings and locomotives, Al-Ula views and oasis and Mada'in Saleh's views. It also shows the tombs will old grafittis (carving and inscriptions now removed).",
  'inscription_credit_line': 'Royal Commission for AlUla ; اﻟﻌﻼ ﻟﻣﺣﺎﻓظﺔ اﻟﻣﻠﻛﯾﺔ اﻟﮭﯾﺋﺔ',
  'object_count': '1',
  'images': ['output\\images\\RCU.2018.1.12_1.png'],
  'source': 'RCU AlUla Collections - 100 Objects'},
 {'inv_no': 'RCU.2018.2.1',
  'object_title': 'Mission Archéologique en Arabie I',
  'artists_makers': 'Antonin Jaussen Raphaël  Savignac',
  'dimensions': None,
  'material': 'paper (fiber product)',
  'date': '1907 AD',
  'description': '-> e

In [18]:
lc_docs

[Document(metadata={'inv_no': 'RCU.2018.1.12', 'title': "Mada'in Salih", 'artist': 'Andreas Schmidt-Colinet', 'dimensions': None, 'date': '1982 AD', 'material': 'reversal film; Material: plastic (material)', 'inscription_credit_line': 'Royal Commission for AlUla ; اﻟﻌﻼ ﻟﻣﺣﺎﻓظﺔ اﻟﻣﻠﻛﯾﺔ اﻟﮭﯾﺋﺔ', 'object_count': '1', 'images': ['output\\images\\RCU.2018.1.12_1.png'], 'source': 'RCU AlUla Collections - 100 Objects'}, page_content="Inv. No.: RCU.2018.1.12 Title: Mada'in Salih Artists / Makers: Andreas Schmidt-Colinet Material: reversal film; Material: plastic (material) Date: 1982 AD Object Count: 1 Inscription Credit Line: Royal Commission for AlUla ; اﻟﻌﻼ ﻟﻣﺣﺎﻓظﺔ اﻟﻣﻠﻛﯾﺔ اﻟﮭﯾﺋﺔ Description: -> en: From Dr. A. Schmidt-Colinet collection of 141 colour 35mm slides from 1982 Hegra Excavations. It shows Mad'in Saleh tombs, Hejaz railway buildings and locomotives, Al-Ula views and oasis and Mada'in Saleh's views. It also shows the tombs will old grafittis (carving and inscriptions now removed).

In [19]:
with open(os.path.join(OUTPUT_DIR, "objects.json"), "w", encoding="utf-8") as f:
        json.dump(objects, f, indent=2, ensure_ascii=False)

In [20]:
with open(os.path.join(OUTPUT_DIR, "langchain_docs.json"), "w", encoding="utf-8") as f:
        json.dump(
            [
                {
                    "content": d.page_content,
                    "metadata": d.metadata
                }
                for d in lc_docs
            ],
            f,
            indent=2,
            ensure_ascii=False
        )

In [21]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
import pickle

embeddings = HuggingFaceEmbeddings(
        model_name="intfloat/multilingual-e5-base",
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True}
    )

d:\MusaddiqueHussainLabs\rcu_heritage_chatbot_demo\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
from uuid import uuid4
import chromadb
from langchain_chroma import Chroma

text_vector_store = Chroma(
    collection_name="heritage_text",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

client = chromadb.PersistentClient(path="./chroma_langchain_db")

uuids = [str(uuid4()) for _ in range(len(lc_docs))]


In [23]:
text_vector_store.add_documents(documents=lc_docs, ids=uuids)

['b058bffa-a67b-4f4f-b206-934b9cab9142',
 '486f743c-8455-4ac0-a1ed-99beaad4b873',
 '00f06f65-f45f-45a1-876b-c441684bb396',
 '46da40cf-a7fc-48ec-9aad-bf49238c8a00',
 'ac72e290-c532-42f8-993b-483155dd5bb2',
 '9439b124-d316-4845-a287-0dba3748807c',
 '75c5750a-d304-4fbf-ac24-e97cc9770ad8',
 '935d44c5-5915-4193-a268-369a63d7bbf0',
 'eece6bea-1ff0-4acf-8a92-6a2c131e3ae5',
 '2ec85e6f-cbb1-4ee9-b277-44d9eb0fedc6',
 '8d9bb96d-3522-4fc3-b253-47954d9a1050',
 '47f16f51-ff6d-480c-8651-54d2ae83e1d5',
 '8b91367e-5bfc-41d4-81c4-5deae1a71799',
 'f1eb6b10-114c-4bc5-adb5-d5e6a51cede2',
 'a96afcfb-d2c0-4760-a0cb-94afd76a0807',
 '6bb24189-bbd7-4bb4-afe8-e2f37a110b63',
 'da78aa4f-5974-4366-972f-e2d97b72c222',
 '4ebed5db-9f1e-44e3-b22e-d39bd95d3288',
 '2cf0e7da-2bbd-464e-ad3c-64282e0653c9',
 'b9142529-d826-4340-b136-425754640dd9',
 '5cf5462b-ed8b-4582-afd7-c790d1ebba20',
 '21aea299-bec6-44d6-89af-704b81d59fc6',
 '417ef378-e539-4f8c-bd1e-efd93ef86e45',
 '0ae5faed-67ad-403a-a1dd-9c4f46acbf6d',
 '8a3ab93e-3459-

In [24]:
results = text_vector_store.similarity_search(
    "RCU.2018.1.12",
    k=2,
    filter={"inv_no": {"$eq": "RCU.2018.1.12"}},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Inv. No.: RCU.2018.1.12 Title: Mada'in Salih Artists / Makers: Andreas Schmidt-Colinet Material: reversal film; Material: plastic (material) Date: 1982 AD Object Count: 1 Inscription Credit Line: Royal Commission for AlUla ; اﻟﻌﻼ ﻟﻣﺣﺎﻓظﺔ اﻟﻣﻠﻛﯾﺔ اﻟﮭﯾﺋﺔ Description: -> en: From Dr. A. Schmidt-Colinet collection of 141 colour 35mm slides from 1982 Hegra Excavations. It shows Mad'in Saleh tombs, Hejaz railway buildings and locomotives, Al-Ula views and oasis and Mada'in Saleh's views. It also shows the tombs will old grafittis (carving and inscriptions now removed). [{'source': 'RCU AlUla Collections - 100 Objects', 'date': '1982 AD', 'artist': 'Andreas Schmidt-Colinet', 'inscription_credit_line': 'Royal Commission for AlUla ; اﻟﻌﻼ ﻟﻣﺣﺎﻓظﺔ اﻟﻣﻠﻛﯾﺔ اﻟﮭﯾﺋﺔ', 'images': ['output\\images\\RCU.2018.1.12_1.png'], 'inv_no': 'RCU.2018.1.12', 'material': 'reversal film; Material: plastic (material)', 'title': "Mada'in Salih", 'object_count': '1'}]


In [25]:
import os
import torch
import open_clip
import chromadb
from PIL import Image
from tqdm import tqdm

In [26]:
CHROMA_PATH = "./chroma_langchain_db"
COLLECTION_NAME = "heritage_images"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [27]:
clip_model, _, preprocess = open_clip.create_model_and_transforms(
    "ViT-B-32",
    pretrained="laion2b_s34b_b79k"
)

In [28]:
clip_model = clip_model.to(DEVICE)
clip_model.eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine

In [29]:
image_collection = client.get_or_create_collection(
    name=COLLECTION_NAME
)

In [30]:
def embed_image(image_path):
    image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        features = clip_model.encode_image(image)

    features /= features.norm(dim=-1, keepdim=True)

    return features.cpu().numpy()[0]

In [31]:
def embed_text_for_image_search(text):
    tokens = open_clip.tokenize([text]).to(DEVICE)

    with torch.no_grad():
        features = clip_model.encode_text(tokens)

    features /= features.norm(dim=-1, keepdim=True)

    return features.cpu().numpy()[0]

In [32]:
def clean_metadata(metadata_dict):
    cleaned = {}

    for k, v in metadata_dict.items():
        if v is None:
            cleaned[k] = ""
        elif isinstance(v, (str, int, float, bool)):
            cleaned[k] = v
        else:
            cleaned[k] = str(v)

    return cleaned


In [33]:
def index_images(objects):

    for obj in tqdm(objects, desc="Indexing Images"):

        for idx, img_path in enumerate(obj["images"]):

            if not os.path.exists(img_path):
                continue

            embedding = embed_image(img_path)

            metadata = clean_metadata({
                "inv_no": obj["inv_no"],
                "title": obj.get("object_title"),
                "material": obj.get("material"),
                "date": obj.get("date"),
                "image_path": img_path,
                "source": obj.get("source")
            })

            image_collection.add(
                ids=[f"{obj['inv_no']}_{idx}"],
                embeddings=[embedding.tolist()],
                metadatas=[metadata]
            )

    print("Image indexing completed.")


In [34]:
def search_similar_image(query_image_path, top_k=3):

    embedding = embed_image(query_image_path)

    results = image_collection.query(
        query_embeddings=[embedding.tolist()],
        n_results=top_k
    )

    return results


# ==============================
# TEXT → IMAGE SEARCH
# ==============================

def search_image_by_text(query_text, top_k=3):

    embedding = embed_text_for_image_search(query_text)

    results = image_collection.query(
        query_embeddings=[embedding.tolist()],
        n_results=top_k
    )

    return results

In [35]:
index_images(objects)

Indexing Images: 100%|██████████| 107/107 [00:09<00:00, 11.54it/s]

Image indexing completed.


In [36]:
results = search_image_by_text("RCU.2018.1.12", top_k=3)

results["metadatas"]

[[{'date': '',
   'source': 'RCU AlUla Collections - 100 Objects',
   'title': 'Pottery Vessel',
   'image_path': 'output\\images\\RCU.2019.65_1.png',
   'material': 'ceramic (material)',
   'inv_no': 'RCU.2019.65'},
  {'date': '',
   'inv_no': 'RCU.2020.282',
   'image_path': 'output\\images\\RCU.2020.282_1.png',
   'material': '',
   'source': 'RCU AlUla Collections - 100 Objects',
   'title': 'Traditional frying pan made from mud to fry Sambusa'},
  {'source': 'RCU AlUla Collections - 100 Objects',
   'date': '',
   'material': 'ceramic (material)',
   'inv_no': 'RCU.2020.147.2675',
   'title': 'Pottery Jar',
   'image_path': 'output\\images\\RCU.2020.147.2675_1.png'}]]